In [ ]:
from __future__ import print_function

import tensorflow as tf
import os
from google.colab import drive
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
from PIL import Image , ImageOps ,ImageFilter
from keras.utils import np_utils
import os
import matplotlib
import numpy as np
from numpy  import *
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
import cv2
drive.mount('/content/gdrive')
from google.colab.patches import cv2_imshow
from IPython.display import clear_output
# Dataset Parameters - CHANGE HERE
MODE = 'folder' # or 'file', if you choose a plain text file (see above).
DATASET_PATH = '/content/gdrive/MyDrive/dataset majaor/' # the dataset file or root folder path.
savepath = '/content/gdrive/MyDrive/d1'
# Image Parameters
N_CLASSES = 115 # CHANGE HERE, total number of classes
IMG_HEIGHT = 64 # CHANGE HERE, the image height to be resized to
IMG_WIDTH = 64 # CHANGE HERE, the image width to be resized to
CHANNELS = 3 # The 3 color channels, change to 1 if grayscale
learning_rate = 0.001
num_steps = 10000
batch_size = 128
display_step = 100

# Network Parameters
dropout = 0.75 # Dropout, probability to keep units

Mounted at /content/gdrive


In [ ]:
def read_images123(dataset_path, mode, batch_size):
    imagepaths, labels = list(), list()
    temp=0
    if mode == 'file':
        # Read dataset file
        data = open(dataset_path, 'r').read().splitlines()
        for d in data:

            imagepaths.append(d.split(' ')[0])
            labels.append(int(d.split(' ')[1]))

    elif mode == 'folder':
        # An ID will be affected to each sub-folders by alphabetical order
        label = ''
        # List the directory
        try:  # Python 2
            classes = sorted(os.walk(dataset_path).next()[1])
        except Exception:  # Python 3
            classes = sorted(os.walk(dataset_path).__next__()[1])
        # List each sub-directory (the classes)
        for c in classes:
            c_dir = os.path.join(dataset_path, c)
            try:  # Python 2
                walk = os.walk(c_dir).next()
            except Exception:  # Python 3
                walk = os.walk(c_dir).__next__()
            # Add each image to the training set
            for sample in walk[2]:
                # Only keeps jpeg images
                if sample.endswith('.jpg') or sample.endswith('.jpeg'):
                    #f = open(os.path.join(c_dir, sample))
                    label = c_dir.split('/')[5]
                    a=int(label)
                    if a>=10 and a<=56:
                      imagepaths.append(os.path.join(c_dir, sample))
                      labels.append(int(label)-10)
                      temp+=1
                      clear_output()
                      print(temp)
                    else:
                      break;
            #label += 1
    else:
        raise Exception("Unknown mode.")

    #print(imagepaths)
    print(imagepaths)
    print(labels)
    ima=array([array(Image.open(list1)) for list1 in imagepaths],'f')
    #ima,labels=shuffle(ima,labels,random_state=2)

    img=ima[100]
    #plt.imshow(img)

    X_train,X_test,y_train,y_test=train_test_split(ima,labels,test_size=0.2)
    X_train.shape
    X_train = X_train.astype('float32')
    X_test = X_test.astype('float32')
    ## making the paixel values in range 0 to 1
    X_train /= 255
    X_test /= 255
    print('X_train shape:', X_train.shape)
    print(X_train.shape[0], 'train samples')
    print(X_test.shape[0], 'test samples')

    ##categorized the output in 10 distinct vales
    y_train = keras.utils.to_categorical(y_train, 47)
    y_test = keras.utils.to_categorical(y_test, 47)


    X_train = X_train.reshape(X_train.shape[0], 200, 200, 1)
    X_test = X_test.reshape(X_test.shape[0], 200, 200, 1)
    #num_classes  10
    #epochs  10
    input_shape=(200,200,1)

    model = tf.keras.models.Sequential([
    # Note the input shape is the desired size of the image 150x150 with 3 bytes color
    # This is the first convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=input_shape),
    tf.keras.layers.MaxPooling2D(2, 2),
    # The second convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The third convolution
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The fourth convolution
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),
    # 512 neuron hidden layer
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(47, activation='softmax')
    ])


    """
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(5, 5),activation='relu',input_shape=input_shape))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(64, (5, 5), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    #model.add(Dropout(0.2))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(15, activation='softmax'))
    """
    model.compile(loss=keras.losses.categorical_crossentropy,optimizer='rmsprop',metrics=['accuracy'])
    model.summary()
    history = model.fit(X_train, y_train,epochs=50,batch_size=100,verbose=1,validation_data=(X_test, y_test))
    print("The model has successfully trained")
    print(history.history.keys())
    # summarize history for accuracy
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()
    # summarize history for loss
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()
    score = model.evaluate(X_test, y_test, verbose=0)
    print('Test loss:', score[0])
    print('Test accuracy:', score[1])
    model.save(r'/content/gdrive/MyDrive/alltest_model_10_56_50epochs_Mam_submit_run.h5') ##path where you want to save the model
    print('alltest_model_10_56_50epochs.h5')



In [ ]:
savepath='/content/gdrive/MyDrive/d7'
read_images123(savepath, MODE, batch_size)

6122
['/content/gdrive/MyDrive/d7/10/A_45.jpg', '/content/gdrive/MyDrive/d7/10/A_56.jpg', '/content/gdrive/MyDrive/d7/10/A_67.jpg', '/content/gdrive/MyDrive/d7/10/A_78.jpg', '/content/gdrive/MyDrive/d7/10/A_89.jpg', '/content/gdrive/MyDrive/d7/10/A_100.jpg', '/content/gdrive/MyDrive/d7/10/A_111.jpg', '/content/gdrive/MyDrive/d7/10/A_122.jpg', '/content/gdrive/MyDrive/d7/10/A_2.jpg', '/content/gdrive/MyDrive/d7/10/A_13.jpg', '/content/gdrive/MyDrive/d7/10/A_39.jpg', '/content/gdrive/MyDrive/d7/10/A_40.jpg', '/content/gdrive/MyDrive/d7/10/A_41.jpg', '/content/gdrive/MyDrive/d7/10/A_42.jpg', '/content/gdrive/MyDrive/d7/10/A_43.jpg', '/content/gdrive/MyDrive/d7/10/A_44.jpg', '/content/gdrive/MyDrive/d7/10/A_46.jpg', '/content/gdrive/MyDrive/d7/10/A_47.jpg', '/content/gdrive/MyDrive/d7/10/A_48.jpg', '/content/gdrive/MyDrive/d7/10/A_49.jpg', '/content/gdrive/MyDrive/d7/10/A_50.jpg', '/content/gdrive/MyDrive/d7/10/A_51.jpg', '/content/gdrive/MyDrive/d7/10/A_52.jpg', '/content/gdrive/MyDrive/d